## Демо 1 - Kafka

1) Запускаем kafka
```bash
docker-compose up -d
```

2) Напишем consumer
```python
import json
from kafka import KafkaConsumer

KAFKA_HOST = "localhost"
KAFKA_TOPIC_NAME = "service_backend_feed"
KAFKA_CONSUMER_GROUP = "backend"

consumer = KafkaConsumer(
    bootstrap_servers=f"{KAFKA_HOST}:9092",
    value_deserializer=lambda v: json.loads( v.decode("utf-8") ),
    auto_offset_reset="earliest",
    group_id=KAFKA_CONSUMER_GROUP
)
consumer.subscribe(topics=KAFKA_TOPIC_NAME)

try:
    for message in consumer:
        value = message.value
        print(value)
except Exception as e:
        print("Closing consumer due to error")
        consumer.close()
        raise e
finally:
    print("Closing consumer due to finish")
    consumer.close()
```

3) Запускаем consumer
```bash
python3 consumer.py
```

4) Напишем producer

In [1]:
import json
from datetime import datetime
from kafka import KafkaProducer

In [2]:
KAFKA_HOST = "localhost"
KAFKA_TOPIC_NAME = "service_backend_feed"

In [4]:
# Подробнее:
# https://kafka-python.readthedocs.io/en/master/apidoc/KafkaProducer.html

class MessageProducer:
    broker = None
    topic = None
    producer = None

    def __init__(self, broker, topic):
        self.broker = broker
        self.topic = topic
        self.producer = KafkaProducer(
            bootstrap_servers=self.broker,
            value_serializer=lambda v: json.dumps(v).encode("utf-8"),
            # acks = 0 - после вызова send сообщение считается отправленым
            # acks = 1 - сообщение считается отправленым после получения подтверждения от лидера
            # acks = "all" - сообщение считается отправленым после получения подтверждения от всех реплик
            acks="all",
            retries = 3
        )

    def send_msg(self, msg):
        future = self.producer.send(self.topic, msg)
        self.producer.flush()

5) Запускаем producer

In [5]:
kafka_client = MessageProducer(
    f"{KAFKA_HOST}:9092",
    KAFKA_TOPIC_NAME
)

In [6]:
kafka_client.send_msg({
    "created_at": datetime.utcnow().isoformat(),
    "lol": "kek",
    "4eburek": ["a", "b", "c"]
})

In [10]:
kafka_client.send_msg({
    "created_at": datetime.utcnow().isoformat(),
    "lol": "4",
    "4eburek": ["a", "b", "c"]
})

In [13]:
for i in range(10):
    kafka_client.send_msg({
        "created_at": datetime.utcnow().isoformat(),
        "lol": "4",
        "kek": i,
        "4eburek": ["a", "b", "c"]
    })